In [1]:
# create a class for matching prompt to book
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from transformers import BertTokenizer, BertModel
import torch
from bs4 import BeautifulSoup
import multiprocessing

/Users/iqraimtiaz/miniconda3/envs/environ/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

# define a function to apply the get_text() method to each element
def html_to_string(html):
    try:
        return BeautifulSoup(html, 'html.parser').get_text()
    except TypeError:
        return ""

df = pd.read_csv('/Users/iqraimtiaz/Documents/duke/Courses/ind-study/E-commerce-Product-Reccommendation/data/wc-product-export-27-4-2023-1682611625064.csv')

with multiprocessing.Pool(processes=multiprocessing.cpu_count()) as pool:
    df['Description'] = pool.map(html_to_string, df['Description'])
df['Description'] = df['Description'].str.replace(r'[^a-zA-Z0-9\s]', '').str.replace('\rn', '').str.replace('\r', '').str.replace('\n', '').str.replace('\t', '').str.replace('  ', ' ')

df.to_csv('/Users/iqraimtiaz/Documents/duke/Courses/ind-study/E-commerce-Product-Reccommendation/data/products.csv')


/Users/iqraimtiaz/miniconda3/envs/environ/lib/python3.7/site-packages/ipykernel_launcher.py:4: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  after removing the cwd from sys.path.
/Users/iqraimtiaz/miniconda3/envs/environ/lib/python3.7/site-packages/ipykernel_launcher.py:12: FutureWarning: The default value of regex will change from True to False in a future version.
  if sys.path[0] == "":


In [3]:
df

,ID,Type,SKU,Name,Published,Is featured?,Visibility in catalog,Short description,Description,Date sale price starts,...,Attribute 1 visible,Attribute 1 global,Attribute 2 name,Attribute 2 value(s),Attribute 2 visible,Attribute 2 global,Attribute 3 name,Attribute 3 value(s),Attribute 3 visible,Attribute 3 global
0,20,variable,NaN,FRP Economy Window Shade,1,0,visible,FRP Economy Window Shade with a slight front i...,FRP Window shades comes in various designs and...,NaN,...,1.0,1.0,WS-Sizes,"L-108″ x W-20″ x H-6″, L-120″ x W-20″ x H-6″, ...",1.0,1.0,NaN,NaN,NaN,NaN
1,141,simple,NaN,FRP Plain Sheet,1,0,visible,F.R.P plain sheets are made from a composite m...,Properties Lightweight Easier to transport han...,NaN,...,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,145,simple,NaN,FRP Small Trapezoid Sheet,1,0,visible,FRP small trapezoid sheets are a modern roofin...,Properties Lightweight FRP sheets are lightwei...,NaN,...,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,146,simple,NaN,FRP Small Corrugated Sheet SC,1,0,visible,F.R.P Small Corrugated sheets can be distingui...,Properties Prevents water build up Due to the ...,NaN,...,1.0,1.0,Thickness,"1.5 MM, 2 MM, 2.5 MM, 3 MM",1.0,1.0,NaN,NaN,NaN,NaN
4,148,simple,NaN,FRP Large Corrugated Sheet AC,1,0,visible,FRP large corrugated sheets are an innovative ...,Properties Prevents water build up Due to the ...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160,1248,variation,NaN,Fiber Satellite Dish Master Viva Royal,1,0,visible,NaN,,NaN,...,NaN,1.0,Fiber Dish - Sizes,8ft,NaN,1.0,NaN,NaN,NaN,NaN
161,1249,variation,NaN,Fiber Satellite Dish Master Viva Royal,1,0,visible,NaN,,NaN,...,NaN,1.0,Fiber Dish - Sizes,10ft,NaN,1.0,NaN,NaN,NaN,NaN
162,1250,variable,NaN,Fiber Satellite Dish Master Plus,1,0,visible,The Fiber Satellite Dish Master Plus is the ul...,AdvantagesDAHighQuality ReceptionLess signal i...,NaN,...,1.0,1.0,Fiber Dish - Sizes,"6ft, 8ft",1.0,1.0,NaN,NaN,NaN,NaN
163,1251,variation,NaN,Fiber Satellite Dish Master Plus,1,0,visible,NaN,,NaN,...,NaN,1.0,Fiber Dish - Sizes,6ft,NaN,1.0,NaN,NaN,NaN,NaN


In [12]:
prompt = "looking for a product to cover my three cars in the driveway"

vectorizer = TfidfVectorizer()
prompt_vector = vectorizer.fit_transform([prompt])

for col in ['Name', 'Short description', 'Description']:
    cosine_similarity_scores = []
    for desc in df[col]:
        if isinstance(desc, str):
            desc_vector = vectorizer.transform([desc])
            similarity = cosine_similarity(prompt_vector, desc_vector)[0][0]
        else:
            # If summary is NaN, set similarity score to NaN
            similarity = np.nan
        cosine_similarity_scores.append(similarity)

    df['cs'+col] = cosine_similarity_scores

In [13]:
df[['Name','Short description','Description','csName','csShort description','csDescription']].sort_values(by=['csShort description','csDescription','csName'], ascending=False)
#df[['Name','Short description','Description','cosine_similarity']].sort_values(by='cosine_similarity', ascending=False)

,Name,Short description,Description,csName,csShort description,csDescription
9,FRP Supreme Window Shade,The F.R.P Supreme window shade is a high-quali...,CodeLength Width Height Price 3WS3148 22123WS3...,0.0,0.603023,0.568535
11,FRP Round Up Window Shade,The F.R.P Round up window shade is a durable a...,CodeLength Width Height Price 3WS5148 24243WS5...,0.0,0.572078,0.568535
12,FRP Empire Window Shade,F.R.P Empire Window Shades are a modern and st...,FRP Window shades comes in various designs and...,0.0,0.569803,0.568535
98,FRP Master Plus Wall Shades,The Fiber Reinforced Polymer used in the const...,Properties Durability Easy to Clean Nonporous ...,0.0,0.568535,0.301511
7,FRP Deluxe Window Shade,"FRP Deluxe Window Shade is a high-quality, dur...",FRP Window shades comes in various designs and...,0.0,0.522233,0.568535
...,...,...,...,...,...,...
160,Fiber Satellite Dish Master Viva Royal,NaN,,0.0,NaN,0.000000
161,Fiber Satellite Dish Master Viva Royal,NaN,,0.0,NaN,0.000000
162,Fiber Satellite Dish Master Plus,NaN,,0.0,NaN,0.000000
163,Fiber Satellite Dish Master Plus,NaN,,0.0,NaN,0.000000


In [14]:
df.columns

Index(['ID', 'Type', 'SKU', 'Name', 'Published', 'Is featured?',
       'Visibility in catalog', 'Short description', 'Description',
       'Date sale price starts', 'Date sale price ends', 'Tax status',
       'Tax class', 'In stock?', 'Stock', 'Low stock amount',
       'Backorders allowed?', 'Sold individually?', 'Weight (kg)',
       'Length (in)', 'Width (in)', 'Height (in)', 'Allow customer reviews?',
       'Purchase note', 'Sale price', 'Regular price', 'Categories', 'Tags',
       'Shipping class', 'Images', 'Download limit', 'Download expiry days',
       'Parent', 'Grouped products', 'Upsells', 'Cross-sells', 'External URL',
       'Button text', 'Position', 'Attribute 1 name', 'Attribute 1 value(s)',
       'Attribute 1 visible', 'Attribute 1 global', 'Attribute 2 name',
       'Attribute 2 value(s)', 'Attribute 2 visible', 'Attribute 2 global',
       'Attribute 3 name', 'Attribute 3 value(s)', 'Attribute 3 visible',
       'Attribute 3 global', 'csName', 'csShort descript